# Analyse de données structurées avec SparkSQL

Pour éviter d'avoir à constamment manipuler des jeux de données au format texte, il peut être intéressant de structurer nos données. SparkSQL permet de structurer un jeu de données en définissant un schéma.

1- **EXERCICE** Commencez d'abord par importer le module Python pour Spark et créez un contexte. **Attention**, vérifiez s'il existe déjà un contexte Spark ou gérez les exceptions en conséquence.

2- On importe ensuite les composantes dont on a besoin du module SparkSQL:  
- `SQLContext`: contexte qui va nous permettre d'interroger nos données avec des commandes SQL
- `Row`: classe qui va nous permettre de définir le schéma de données.

In [ ]:
from pyspark.sql import SQLContext, Row
sqlCtx = SQLContext(sc)

3- **EXERCICE** On va créer un nouveau RDD à partir du jeu de données de l'introduction soit `data/pagecounts`. Créez un nouveau RDD à partir de `data/pagecounts`. Transformez ensuite le RDD en un second nommé `parts` où chaque ligne est subdivisée en une liste de 4 éléments.

4.1- **EXERCICE** Pour vérifier que le résultat est correct, comptez le nombre d'éléments dans le RDD `parts`

4.2- **EXERCICE** Affichez les 8 premiers éléments

5- On peut maintenant structurer nos données. Pour ce faire, on va transformer chaque liste dans notre RDD `parts` en un objet `Row`. La structure nous permet de convertir une chaîne de caractères en un type (un entier par exemple) et de nommer les champs de notre structure.

In [ ]:
pagecounts = parts.map(lambda p: Row(lang=p[0], name=p[1], pagecount=long(p[2]), size=long(p[3])))

5.1- **EXERCICE** Afficher l'attribut `name` du premier élément du RDD `pagecounts`.

6- À partir du RDD `pagecounts` où chaque élément est un objet de type `Row`, on peut créer un nouveau RDD de type `DataFrame`. En plus de permettre les opérations standards des RDD, un `DataFrame` contient certaines informations supplémentaires tel que le nom et le type des colonnes du jeu de données.

On commence donc par créer un `DataFrame` à partir d'un RDD existant. La fonction `createDataFrame` détermine automatiquement la structure que devra avoir notre RDD, soit le nom et le type des colonnes, à partir des objets de type `Row`.

In [ ]:
dfPageCounts = sqlCtx.createDataFrame(pagecounts)

In [ ]:
dfPageCounts.show()

L'utilisation de la classe `Row` nous permet de spécifier le type de chacune des colonnes. Si on avait simplement créer un dataframe à partir de `parts`, voici ce qu'on aurait observé.

In [ ]:
sqlCtx.createDataFrame(parts)

7- On enregistre ensuite notre schéma comme une table qui pourra être interrogée à partir de requêtes SQL.

In [ ]:
dfPageCounts.registerTempTable("page_table")

8- On peut maintenant interroger notre jeu de données à l'aide d'une requête SQL.

In [ ]:
top_spanish = sqlCtx.sql("SELECT name, pagecount "
                         "FROM page_table "
                         "WHERE pagecount>=100 "
                         "AND lang='es' "
                         "ORDER BY pagecount DESC "
                         "LIMIT 10")

Décomposons la requête en composantes fonctionelles
### SELECT

Indique les variables que l'on veut collecter. Les nom des variables ont été définis lors de la création des objets `Row`.

### FROM

Indique la source des données. Le nom de la table a été défini lors de l'enregistrement à l'étape 7.

### WHERE

Filtre les entrées selon certaines caractériques. 

### ORDER BY [...] DESC

Indique que l'on veut ordonner nos résultats en fonction de l'une des variables. DESC indique qu'on veut que les données soit ordonnées de manière décroissante. 

### LIMIT N 

Conserve que les N premiers entrées de notre requête.


9- **EXERCICE** Comme pour toute méthode de transformation d'un RDD, l'évaluation de la commande ne s'effectue que lorsqu'un résultat est exigé par une méthode d'action. Entrez la commande d'action permettant de récuperer la totalité des résultats de notre RDD `top_spanish`.

In [ ]:
top_spanish.collect()

On remarque cependant que certaines page reviennent plusieurs fois dans notre palmarès. La raison est qu'on a omis d'additionner le nombre de vues pour une même page. Il faut effectuer une opération d'aggrégation `GROUP` et `SUM`.

In [ ]:
sqlCtx.sql("SELECT name, SUM(pagecount) as sumation "
           "FROM page_table "
           "WHERE pagecount>=100 "
           "AND lang='es' "
           "GROUP BY name "
           "ORDER BY sumation DESC "
           "LIMIT 10").collect()

10- **EXERCICE** Pour vous convaincre de l'utilité de SparkSQL pour simplifier l'analyse de données, écrivez le code nécessaire en utilisant les méthode de transformation des RDD de base (`map`, `filter`, `reduce`, etc.) et d'action (`first`, `collect`, `take`) pour produire le même résultat que la requête SQL précédente.

Utilisez le RDD que vous avez créez au début du notebook comme point de départ.

11- Pour éviter d'avoir à restructer nos données à chaque fois, on peut sauvegarder les au format [Apache Parquet](https://parquet.apache.org/). Le format va conserver le schéma et l'ordre des données intact.

In [ ]:
dfPageCounts.write.parquet("data/pagecounts.parquet")

12- On peut ensuite facilement créer un nouveau `DataFrame` en lisant nos fichiers au format Parquet.

In [ ]:
pagecount_parq = sqlCtx.read.parquet("data/pagecounts.parquet")
pagecount_parq.first()

13- Finalement, on arrête le contexte Spark.

In [ ]:
sc.stop() 

## Informations supplémentaires

- On peut connecter Spark SQL sur une base de données externes, par exemple une base de données PostgreSQL.
- Le `Dataframe` permet d'obtenir le même niveau de performance peu importe le langage utilisée. Les RDD classiques sont normalement plus performant en Java ou en Scala.

## Exercice récapitulatif

1. Créez un nouveu notebook.
1. Créez un nouveau contexte Spark et un contexte SQL.
1. Créez un RDD à partir des données d'entrée `data/pagecounts.parquet`.
2. Transformez le RDD en un RDD contenant la taille totale des pages vues par langue  
    1. À l'aide des méthodes Spark  
    2. À l'aide d'une requête SQL
3. Limitez le contenu du RDD au 3 langues les moins populaires 
4. Affichez le résultat.